<a href="https://colab.research.google.com/github/janbanot/msc-cs-code/blob/main/sem3/DL/DL_2025_Lab5-2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sieci typu transformer

Mechanizm samouwagi (self-attention) zaproponowany w pracy [1] stanowi istotny krok w rozwoju współczesnych sieci neuronowych,
w tym dużych modeli językowych. W bieżącym notatniku rozpatrzymy sieć typu GPT (Generative Pre-trained Transformer)
odpowiadającą wariantowi GPT-2.

[1] Vaswani, Ashish, et al. "Attention is all you need." Advances in neural information processing systems 30 (2017).

In [ ]:
# @title Zwięzła implementacja modelu GPT
# Żródło: https://github.com/karpathy/nanoGPT/blob/master/model.py

!uv pip install torchinfo

import math
import inspect
from dataclasses import dataclass
import random
import numpy as np

import torch
import torch.nn as nn
from torch.nn import functional as F
from torchinfo import summary
from torch.nn.utils.rnn import pad_sequence
import matplotlib.pyplot as plt


class CausalSelfAttention(nn.Module):
    """
    Przyczynowa samouwaga (Causal Self-Attention).
    Token może "patrzeć" tylko na tokeny z przeszłości, tj. za nim, a nie po nim.
    """

    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0

        # Jedna warstwa liniowa liczy naraz Q, K i V (potem rozdzielamy na 3 części).
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd, bias=config.bias)

        # Projekcja wyjściowa po scaleniu głowic.
        self.c_proj = nn.Linear(config.n_embd, config.n_embd, bias=config.bias)

        # Dropout na wyjściu bloku uwagi
        self.resid_dropout = nn.Dropout(config.dropout)
        self.attn_dropout_p = float(config.dropout)

        self.n_head = config.n_head
        self.n_embd = config.n_embd

    def forward(self, x):
        # x: (B, T, C) -> batch, długość sekwencji, wymiar embeddingu
        B, T, C = x.size()
        head_size = C // self.n_head

        # Liczymy Q, K, V i rozdzielamy wynik na trzy tensory.
        q, k, v = self.c_attn(x).split(self.n_embd, dim=2)

        # (B, T, C) -> (B, n_head, T, head_size)
        # .transpose(1, 2) przenosi wymiar głowic przed czas.
        q = q.view(B, T, self.n_head, head_size).transpose(1, 2)
        k = k.view(B, T, self.n_head, head_size).transpose(1, 2)
        v = v.view(B, T, self.n_head, head_size).transpose(1, 2)

        # PyTorch 2.0+: scaled_dot_product_attention może użyć Flash Attention.
        # is_causal=True wymusza maskę trójkątną (brak wglądu w przyszłość).
        y = F.scaled_dot_product_attention(
            q, k, v,
            attn_mask=None,
            dropout_p=self.attn_dropout_p if self.training else 0.0,
            is_causal=True,
        )

        # Scal głowice: (B, n_head, T, head_size) -> (B, T, C)
        y = y.transpose(1, 2).contiguous().view(B, T, C)

        # Projekcja + dropout rezydualny
        return self.resid_dropout(self.c_proj(y))


class MLP(nn.Module):
    """
    Prosty MLP działający niezależnie na każdym tokenie.
    """
    def __init__(self, config):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(config.n_embd, 4 * config.n_embd, bias=config.bias),
            nn.GELU(),
            nn.Linear(4 * config.n_embd, config.n_embd, bias=config.bias),
            nn.Dropout(config.dropout),
        )

    def forward(self, x):
        return self.net(x)


class Block(nn.Module):
    """
    Pojedynczy blok Transformera (pre-norm):
    LN -> Attention -> resid, LN -> MLP -> resid
    """

    def __init__(self, config):
        super().__init__()
        self.ln_1 = nn.LayerNorm(config.n_embd, elementwise_affine=True, bias=config.bias)
        self.attn = CausalSelfAttention(config)
        self.ln_2 = nn.LayerNorm(config.n_embd, elementwise_affine=True, bias=config.bias)
        self.mlp = MLP(config)

    def forward(self, x):
        # Pre-norm: najpierw normalizacja, potem operacja, potem dodanie rezydualne.
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x


@dataclass
class GPTConfig:
    block_size: int = 1024
    vocab_size: int = 50304
    n_layer: int = 12
    n_head: int = 12
    n_embd: int = 768
    dropout: float = 0.0
    bias: bool = True


class GPT(nn.Module):
    def __init__(self, config: GPTConfig):
        super().__init__()
        assert config.vocab_size is not None
        assert config.block_size is not None
        self.config = config

        self.transformer = nn.ModuleDict(dict(
            wte=nn.Embedding(config.vocab_size, config.n_embd),    # embedding tokenów
            wpe=nn.Embedding(config.block_size, config.n_embd),    # embedding pozycji
            drop=nn.Dropout(config.dropout),
            h=nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
            ln_f=nn.LayerNorm(config.n_embd, bias=config.bias),
        ))

        # Głowa językowa: projekcja na rozmiar słownika.
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)

        # Weight tying: wte.weight i lm_head.weight to ten sam parametr.
        # Zmniejsza liczbę parametrów i zwykle poprawia jakość.
        self.transformer.wte.weight = self.lm_head.weight

        # Bufor z indeksami pozycji: unikamy torch.arange w każdym forward().
        # persistent=False => nie zapisuje się do checkpointów (bo można odtworzyć).
        self.register_buffer(
            "pos_idx",
            torch.arange(config.block_size, dtype=torch.long),
            persistent=False,
        )

        # Inicjalizacja wag
        self.apply(self._init_weights)

        # Specjalna inicjalizacja dla projekcji rezydualnych (jak w GPT-2),
        # aby stabilizować wariancję w głębokiej sieci na starcie treningu.
        for pn, p in self.named_parameters():
            if pn.endswith("c_proj.weight"):
                torch.nn.init.normal_(p, mean=0.0, std=0.02 / math.sqrt(2 * config.n_layer))

        print("Liczba parametrów: %.2fM" % (self.get_num_params() / 1e6,))

    def get_num_params(self) -> int:
        """Zwraca liczbę parametrów (bez embeddingów pozycyjnych)."""
        n_params = sum(p.numel() for p in self.parameters())
        n_params -= self.transformer.wpe.weight.numel()
        return n_params

    def _init_weights(self, module: nn.Module) -> None:
        """Domyślna inicjalizacja wag (rozkład normalny)."""
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.01)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.01)

    def forward(self, idx: torch.Tensor, targets: torch.Tensor | None = None):
        device = idx.device
        b, t = idx.size()
        assert t <= self.config.block_size, (
            f"Sekwencja {t} jest dłuższa niż block_size={self.config.block_size}"
        )

        # (b, t, n_embd) + (t, n_embd) => broadcast po batchu
        tok_emb = self.transformer.wte(idx)

        pos = self.pos_idx[:t].to(device)   # Pozycje kolejnych tokenów
        pos_emb = self.transformer.wpe(pos) # Model uczy się kodowania pozycji

        x = self.transformer.drop(tok_emb + pos_emb)  # Opcjonalny dropout

        for block in self.transformer.h:
            x = block(x)
        x = self.transformer.ln_f(x)

        if targets is not None:
            # Trening: logity dla wszystkich pozycji + loss.
            logits = self.lm_head(x)  # (b, t, vocab)

            # cross_entropy oczekuje (N, C, ...) dla logits oraz (N, ...) dla targetów,
            # więc przestawiamy osie na (b, vocab, t).
            loss = F.cross_entropy(
                logits.transpose(1, 2),
                targets,
                ignore_index=-1,  # Ignoruj tokeny o tej wartości
            )
        else:
            # Inferencja: interesuje nas predykcja następnego tokena (ostatnia pozycja).
            logits = self.lm_head(x[:, [-1], :])  # (b, 1, vocab)
            loss = None

        return logits, loss

    def configure_optimizers(self, weight_decay, learning_rate, betas, device_type):
        """
        AdamW z podziałem parametrów na:
        - decay: wagi warstw liniowych (nn.Linear.weight)
        - no_decay: biasy, LayerNorm, Embeddingi
        """

        whitelist = (nn.Linear,)
        blacklist = (nn.LayerNorm, nn.Embedding)

        # mapowanie: id(param) -> (param, czy_decay)
        # czy_decay = True oznacza, że parametr trafi do grupy z weight_decay
        param_to_decay = {}
        id_to_param = {}

        for module in self.modules():
            for name, param in module.named_parameters(recurse=False):
                if not param.requires_grad:
                    continue

                pid = id(param)
                id_to_param[pid] = param

                # Reguły klasyfikacji:
                is_bias = name.endswith("bias")
                is_linear_weight = isinstance(module, whitelist) and name.endswith("weight")
                is_blacklisted = isinstance(module, blacklist)

                # Priorytet: NO_DECAY wygrywa zawsze (szczególnie ważne przy weight tying).
                if is_bias or is_blacklisted:
                    param_to_decay[pid] = False
                elif is_linear_weight:
                    # decay tylko jeśli parametr nie został wcześniej oznaczony jako no_decay
                    param_to_decay.setdefault(pid, True)
                else:
                    # bezpieczny domyślny wybór: brak decay
                    param_to_decay.setdefault(pid, False)

        decay_params = [id_to_param[pid] for pid, dec in param_to_decay.items() if dec]
        nodecay_params = [id_to_param[pid] for pid, dec in param_to_decay.items() if not dec]

        optim_groups = [
            {"params": decay_params, "weight_decay": weight_decay},
            {"params": nodecay_params, "weight_decay": 0.0},
        ]

        # device_type: jeśli przekazujesz torch.device, użyj device_type = device.type
        if isinstance(device_type, torch.device):
            device_type = device_type.type

        use_fused = (device_type == "cuda") and ("fused" in inspect.signature(torch.optim.AdamW).parameters)
        print(f"Używanie fused AdamW: {use_fused}")

        return torch.optim.AdamW(
            optim_groups,
            lr=learning_rate,
            betas=betas,
            fused=use_fused,
        )

    @torch.no_grad()
    def generate(self, idx: torch.Tensor, max_new_tokens: int, temperature: float = 1.0):
        """
        Generuje kolejne tokeny autoregresyjnie.

        Ważne:
        - Zapamiętujemy poprzedni tryb (train/eval) i przywracamy go na końcu,
          żeby generate() nie psuło treningu, jeśli zostanie wywołane w trakcie.
        - @torch.no_grad() wyłącza gradienty (szybciej i mniej pamięci).
        """
        was_training = self.training
        self.eval()
        try:
            for _ in range(max_new_tokens):
                # Ograniczamy kontekst do block_size (model nie widzi dalej).
                idx_cond = idx[:, -self.config.block_size:]

                # forward() w trybie inferencji zwraca logity tylko dla ostatniej pozycji (b, 1, vocab)
                logits, _ = self(idx_cond)
                logits = logits[:, -1, :]  # (b, vocab)

                # temperature=0 -> deterministycznie (argmax)
                if temperature == 0.0:
                    idx_next = torch.argmax(logits, dim=-1, keepdim=True)
                else:
                    logits = logits / temperature
                    probs = F.softmax(logits, dim=-1)
                    idx_next = torch.multinomial(probs, num_samples=1)

                idx = torch.cat([idx, idx_next], dim=1)

            return idx
        finally:
            # Przywróć poprzedni tryb modelu.
            self.train(was_training)

## Przykład 1.

Spróbujmy wytrenować model GPT rozwiązujący problem odwracania słowa
dla ciągów bitowych, np.

    0101 -> 1010
    110 -> 011

Dane kodowane będą za pomocą dodatkowych symboli, tj. separatora `#`, końca ciągu `<EOS>` wypełnienia (padding) `<PAD>`.

In [ ]:
# @title Generowanie danych treningowych

# Słownik tokenów: 0/1/# oraz znaczniki specjalne PAD (do wypełniania) i EOS (koniec sekwencji)
VOCAB = {'0': 0, '1': 1, '#': 2, '<PAD>': 3, '<EOS>': 4}
INV_VOCAB = {v: k for k, v in VOCAB.items()}  # przydatne do debugowania/drukowania
PAD_IDX = VOCAB['<PAD>']
EOS_IDX = VOCAB['<EOS>']

def bin_rev_encode(tokens):
    """Zamienia listę tokenów (np. ['0','1','#','1','0','<EOS>']) na listę id."""
    return [VOCAB[t] for t in tokens]

def bin_rev_decode(tokens):
    return ''.join(INV_VOCAB[t] for t in tokens)

def bin_rev_make_example(n_bits):
    """
    Generuje pojedynczy przykład:
      bits  = losowy ciąg bitów (np. '01')
      rev   = odwrócony ciąg (np. '10')
      tokens= ['0','1','#','1','0','<EOS>']
    Zwraca tensor z identyfikatorami tokenów,
        bits + '#' oraz rev do ewentualnego podglądu/debugowania.
    """
    bits = ''.join(random.choice('01') for _ in range(n_bits))
    rev = bits[::-1]
    tokens = list(bits) + ['#'] + list(rev) + ['<EOS>']
    return torch.tensor(bin_rev_encode(tokens), dtype=torch.long), bits + '#', rev

In [ ]:
# @title Funkcje evaluacji oraz treningu

def eval_accuracy(model, max_bits, make_example, decode, eval_examples=100, n_show=1):
    """
    Oblicza i zwraca:
    - sequence accuracy -- dokładność (ile przykładów poprawnych)
    - token-level accuracy (ile tokenów w sufiksie po '#' się zgadza)
    """
    seq_correct = 0
    seq_total = 0

    tok_correct = 0
    tok_total = 0

    shown = 0

    device = next(model.parameters()).device

    for _ in range(eval_examples):
        ids, prefix, suffix = make_example(random.randint(1, max_bits))

        cut = len(prefix)
        prompt = ids[:cut].unsqueeze(0).to(device)
        max_new = int(ids.numel() - cut) # ile tokenów trzeba dogenerować

        out = model.generate(prompt, max_new_tokens=max_new, temperature=0.0)

        generated_suffix = out[0, cut:].detach().cpu()
        target_suffix = ids[cut:].detach().cpu()

        ok = (generated_suffix.numel() == target_suffix.numel()
              and torch.equal(generated_suffix, target_suffix))
        if ok:
            seq_correct += 1
        seq_total += 1

        # Ile tokenów OK
        L = min(generated_suffix.numel(), target_suffix.numel())
        if L > 0:
            tok_correct += (generated_suffix[:L] == target_suffix[:L]).sum().item()
            tok_total += L

        if shown < n_show:  # Opcjonalny podgląd przykładu
            prompt_str = decode(prompt[0].detach().cpu().tolist())
            gen_str    = decode(out[0].detach().cpu().tolist())
            tgt_str    = decode(ids.detach().cpu().tolist())

            print(f"  [{shown+1}] prompt: {prompt_str}")
            print(f"      gen:   {gen_str}")
            print(f"      tgt:   {tgt_str}")
            print(f"      seq_ok:{ok}    (inp={prefix}, out={out})")
            shown += 1

    seq_acc = (seq_correct / seq_total) if seq_total > 0 else float('nan')
    tok_acc = (tok_correct / tok_total) if tok_total > 0 else float('nan')

    return seq_acc, tok_acc


def train_model(model, make_example, decode, max_bits=15, batch_size=128, total_steps=10_000, best_model_path='best_model.pt'):
    MAX_LR = 1e-3
    eval_every = 500

    device = next(model.parameters()).device

    optimizer = model.configure_optimizers(
        weight_decay=0.01,
        learning_rate=5e-4,
        betas=(0.9, 0.99),
        device_type=device
    )

    scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimizer,
        max_lr=MAX_LR,
        total_steps=total_steps,
        pct_start=0.1
    )

    loss_history = []
    print("--- Rozpoczynanie Treningu ---")
    model.train()

    best_loss = float('inf')

    for step in range(total_steps):
        # Dynamiczne generowanie batcha
        batch = [make_example(random.randint(1, max_bits)) for _ in range(batch_size)]

        examples = [tok_ids for tok_ids, _, _ in batch]

        inputs = [s[:-1] for s in examples]
        labels = [s[1:]  for s in examples]

        # xs: 0, 1, #, 1, 0,     <EOS>, <PAD>, ..., <PAD>
        xs = pad_sequence(inputs, batch_first=True, padding_value=PAD_IDX).to(device)
        # ys: 1, #, 1, 0, <EOS>, <PAD>, ..., <PAD>, -1
        ys = pad_sequence(labels, batch_first=True, padding_value=-1).to(device)

        _, loss = model(xs, targets=ys)

        current_loss = float(loss.item())
        loss_history.append(current_loss)

        if current_loss < best_loss:
            best_loss = current_loss
            torch.save(model.state_dict(), best_model_path)

        if step % eval_every == 0:
            current_lr = scheduler.get_last_lr()[0]
            acc, tok_acc = eval_accuracy(model, max_bits, make_example=make_example, decode=decode)
            print(f"Krok {step}, Loss: {current_loss:.4f}, Acc: {acc*100:.1f}%, Tok. acc: {tok_acc*100:.1f}% LR: {current_lr:.6f}")

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()

    print(f"--- Koniec treningu. Najlepsza strata: {best_loss:.4f} ---")

    # Przywracanie najlepszego modelu
    print(f"Ładowanie najlepszego modelu z {best_model_path}...")
    model.load_state_dict(torch.load(best_model_path))

    acc, tok_acc = eval_accuracy(model, max_bits, make_example=make_example, decode=decode, eval_examples=1000)
    print(f"Final acc: {acc*100:.1f}%, Tok. acc: {tok_acc*100:.1f}%")

    # Wykres
    plt.figure(figsize=(8, 4))
    plt.plot(loss_history)
    plt.xlabel('Krok')
    plt.ylabel('Wartość straty')
    plt.grid(True, alpha=0.3)

    # Dodanie wygładzonej średniej (dla czytelności przy dużych wahaniach)
    if len(loss_history) > 50:
        # Prosta średnia krocząca
        window = 50
        smoothed = [sum(loss_history[i:i+window])/window for i in range(len(loss_history)-window)]
        plt.plot(range(window, len(loss_history)), smoothed, color='red', linewidth=1, label='Średnia krocząca')
        plt.legend()

    plt.show()

    return model

In [ ]:
# Ustawienie seed dla reprodukowalności
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    # Dla pełnej deterministyczności (może spowolnić trening na GPU, ale zapewnia reprodukowalność)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


# Konfiguracja naszego modelu
@dataclass
class BinRevConfig:
    block_size: int = 48   # Maksymalna długość kontekstu (musi pomieścić "bity + # + rewers")
    vocab_size: int = len(VOCAB)    # Rozmiar słownika (0, 1, #, PAD, EOS)
    n_layer: int = 1       # Tylko 1 warstwa (zadanie jest łatwe)
    n_head: int = 4        # Tylko 1 głowica uwagi
    n_embd: int = 64       # Mały wymiar osadzenia
    dropout: float = 0.0   # Brak dropoutu (mały model, dużo danych syntetycznych)
    bias: bool = False     # Wyłączamy bias dla szybkości


set_seed(12383249)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = 'cpu'
print(f"Running on device: {device}")
config = BinRevConfig()
model = GPT(config=config)
model = model.to(device)

# Wyświetl podsumowanie struktury modelu i liczby parametrów
print(summary(model, device=device))

model = train_model(model, make_example=bin_rev_make_example,
                    decode=bin_rev_decode, total_steps=4000)

In [ ]:
prompt = "11111101010#"  # możesz też dać np. "0110#" itd.

idx = torch.tensor(bin_rev_encode(prompt), dtype=torch.long).unsqueeze(0).to(device)   # (1, T)

# Ile tokenów dogenerować? Najprościej: tyle co liczba bitów (tu 2) + ewentualnie 1 na <EOS>
max_new = len(prompt)

out = model.generate(idx, max_new_tokens=max_new, temperature=0.0)  # 0.0 = deterministycznie
completed = bin_rev_decode(out[0].tolist())

print("Wynik:", completed)


## Zad. 1

Na podstawie poprzedniego przykładu wytrenuj model rozwiązujący problem
inkrementacji bitowej dla ciągów o długości do 15 bitów.

Przykładowo, "1011=1100" (11+1=12).